In [1]:
import numpy as np

import pandas as pd

from datetime import datetime as dt

import itertools

In [2]:
# Read data from the CSV into a dataframe

loc = "./Premier-League/"

years = range(2001, 2022)  # Anos de 2000 a 2021
datasets = []

for year in years:
    file_name = f"{loc}{year}.csv"
    dataset = pd.read_csv(file_name)
    datasets.append(dataset)

print("Todos os Datasets Foram lidos.")

Todos os Datasets Foram lidos.


In [3]:
datasets[0].columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW',
       'AHW', 'HC', 'AC', 'HF', 'AF', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')

In [4]:
# Parse data as time 
def parse_date(date):
    if date == '':
        return None
    else:
        if len(date) != 8:
            date_format = '%d/%m/%Y'
        else:
            date_format = '%d/%m/%y'
        return dt.strptime(date, date_format)

    
for i, dataset in enumerate(datasets):
    datasets[i].Date = dataset.Date.apply(parse_date)

print("Datas tipadas Corretamente.")

Datas tipadas Corretamente.


In [5]:
# Gets the goals scored agg arranged by teams and matchweek
def get_goals_scored(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam'):
        teams[i[0]] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG']
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam'):
        teams[i[0]] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []
    
    counting = pd.DataFrame(columns = ["Team", "Counting"])
    counting.Team = playing_stat.HomeTeam.sort_values().unique()
    counting.Counting = 0

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        #display(counting.loc[(counting.Team == ht) |  (counting.Team == at)])
        #print(GS.loc[ht][counting.loc[counting.Team == ht, "Counting"].values[0]])
        #print(GS.loc[at][counting.loc[counting.Team == at, "Counting"].values[0]])
        HTGS.append(GS.loc[ht][counting.loc[counting.Team == ht, "Counting"].values[0]])
        ATGS.append(GS.loc[at][counting.loc[counting.Team == at, "Counting"].values[0]])
        HTGC.append(GC.loc[ht][counting.loc[counting.Team == ht, "Counting"].values[0]])
        ATGC.append(GC.loc[at][counting.loc[counting.Team == at, "Counting"].values[0]])
        
        counting.loc[counting.Team == ht, "Counting"] += 1
        counting.loc[counting.Team == at, "Counting"] += 1
        if ((i + 1)% 10) == 0:
            j = j + 1
    # HTGS -> Home Team Gols Scored
    # ATGC -> Away Team Gols Conceded
    #print(HTGC)
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    print(playing_stat.Date.iloc[-1].year, " OK!")
    return playing_stat

for i, dataset in enumerate(datasets):
    datasets[i] = get_gss(dataset)

print("Finalizou.")

2001  OK!
2002  OK!
2003  OK!
2004  OK!
2005  OK!
2006  OK!
2007  OK!
2008  OK!
2009  OK!
2010  OK!
2011  OK!
2012  OK!
2013  OK!
2014  OK!
2015  OK!
2016  OK!
2017  OK!
2018  OK!
2019  OK!
2020  OK!
2021  OK!
Finalizou.


In [6]:
datasets[1].loc[:38, ["Date","HomeTeam", "AwayTeam","FTHG", "FTAG", "HTGS", "ATGS", "HTGC","ATGC"]].sort_values(by="Date")
# APARANTEMENTE TUDO OK

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTGS,ATGS,HTGC,ATGC
0,2001-08-18,Charlton,Everton,1,2,0,0,0,0
1,2001-08-18,Derby,Blackburn,2,1,0,0,0,0
2,2001-08-18,Leeds,Southampton,2,0,0,0,0,0
3,2001-08-18,Leicester,Bolton,0,5,0,0,0,0
4,2001-08-18,Liverpool,West Ham,2,1,0,0,0,0
5,2001-08-18,Middlesbrough,Arsenal,0,4,0,0,0,0
6,2001-08-18,Sunderland,Ipswich,1,0,0,0,0,0
7,2001-08-18,Tottenham,Aston Villa,0,0,0,0,0,0
9,2001-08-19,Man United,Fulham,3,2,0,0,0,0
8,2001-08-19,Chelsea,Newcastle,1,1,0,0,0,0


In [7]:
# Gets the goals scored agg arranged by teams and matchweek
def get_shoots_on_target(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam'):
        teams[i[0]] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HST = playing_stat.iloc[i]['HST']
        AST = playing_stat.iloc[i]['AST']
        teams[playing_stat.iloc[i].HomeTeam].append(HST)
        teams[playing_stat.iloc[i].AwayTeam].append(AST)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    shoots_on_goal = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    shoots_on_goal[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        shoots_on_goal[i] = shoots_on_goal[i] + shoots_on_goal[i-1]
    return shoots_on_goal

def get_shoots(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam'):
        teams[i[0]] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HS = playing_stat.iloc[i]['HS']
        AS = playing_stat.iloc[i]['AS']
        teams[playing_stat.iloc[i].HomeTeam].append(HS)
        teams[playing_stat.iloc[i].AwayTeam].append(AS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    shoots = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    shoots[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        shoots[i] = shoots[i] + shoots[i-1]
    return shoots

def get_gss(playing_stat):
    ST = get_shoots_on_target(playing_stat)
    S = get_shoots(playing_stat)
   
    j = 0
    HST = []
    AST = []
    HS = []
    AS = []
    
    counting = pd.DataFrame(columns = ["Team", "Counting"])
    counting.Team = playing_stat.HomeTeam.sort_values().unique()
    counting.Counting = 0

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        #display(counting.loc[(counting.Team == ht) |  (counting.Team == at)])
        #print(GS.loc[ht][counting.loc[counting.Team == ht, "Counting"].values[0]])
        #print(GS.loc[at][counting.loc[counting.Team == at, "Counting"].values[0]])
        HST.append(ST.loc[ht][counting.loc[counting.Team == ht, "Counting"].values[0]])
        AST.append(ST.loc[at][counting.loc[counting.Team == at, "Counting"].values[0]])
        HS.append(S.loc[ht][counting.loc[counting.Team == ht, "Counting"].values[0]])
        AS.append(S.loc[at][counting.loc[counting.Team == at, "Counting"].values[0]])
        
        counting.loc[counting.Team == ht, "Counting"] += 1
        counting.loc[counting.Team == at, "Counting"] += 1
        if ((i + 1)% 10) == 0:
            j = j + 1
    # HTGS -> Home Team Gols Scored
    # ATGC -> Away Team Gols Conceded
    #print(HTGC)
    playing_stat['HSTA'] = HST
    playing_stat['ASTA'] = AST
    playing_stat['HSA'] = HS
    playing_stat['ASA'] = AS
    print(playing_stat.Date.iloc[-1].year, " OK!")
    return playing_stat

for i, dataset in enumerate(datasets):
    datasets[i] = get_gss(dataset)

print("Finalizou.")

2001  OK!
2002  OK!
2003  OK!
2004  OK!
2005  OK!
2006  OK!
2007  OK!
2008  OK!
2009  OK!
2010  OK!
2011  OK!
2012  OK!
2013  OK!
2014  OK!
2015  OK!
2016  OK!
2017  OK!
2018  OK!
2019  OK!
2020  OK!
2021  OK!
Finalizou.


In [8]:
datasets[1].loc[:38, ["Date","HomeTeam", "AwayTeam","HS", "AS", "HST", "AST", "HSA", "ASA", "HSTA", "ASTA"]].sort_values(by="Date")

,Date,HomeTeam,AwayTeam,HS,AS,HST,AST,HSA,ASA,HSTA,ASTA
0,2001-08-18,Charlton,Everton,8,12,4,9,0,0,0,0
1,2001-08-18,Derby,Blackburn,7,14,3,4,0,0,0,0
2,2001-08-18,Leeds,Southampton,16,11,6,6,0,0,0,0
3,2001-08-18,Leicester,Bolton,6,18,1,8,0,0,0,0
4,2001-08-18,Liverpool,West Ham,9,3,6,3,0,0,0,0
5,2001-08-18,Middlesbrough,Arsenal,6,14,2,9,0,0,0,0
6,2001-08-18,Sunderland,Ipswich,12,10,6,4,0,0,0,0
7,2001-08-18,Tottenham,Aston Villa,8,11,2,2,0,0,0,0
9,2001-08-19,Man United,Fulham,12,8,6,5,0,0,0,0
8,2001-08-19,Chelsea,Newcastle,13,9,8,3,0,0,0,0


In [9]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points


def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam'):
        teams[i[0]] = []
    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def get_agg_points(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    print(playing_stat.Date.iloc[-1].year, " OK!")
    return playing_stat

for i, dataset in enumerate(datasets):
    datasets[i] = get_agg_points(dataset)

print("Finalizou.")

2001  OK!
2002  OK!
2003  OK!
2004  OK!
2005  OK!
2006  OK!
2007  OK!
2008  OK!
2009  OK!
2010  OK!
2011  OK!
2012  OK!
2013  OK!
2014  OK!
2015  OK!
2016  OK!
2017  OK!
2018  OK!
2019  OK!
2020  OK!
2021  OK!
Finalizou.


In [10]:
datasets[15].loc[300:, ["Date","HomeTeam", "AwayTeam","FTR", "HTP", "ATP"]].sort_values(by="Date")

# Aparentemente OK

,Date,HomeTeam,AwayTeam,FTR,HTP,ATP
300,2016-03-20,Newcastle,Sunderland,D,25,26
301,2016-03-20,Southampton,Liverpool,H,44,45
302,2016-03-20,Tottenham,Bournemouth,H,58,38
309,2016-04-02,Sunderland,West Brom,D,26,39
308,2016-04-02,Stoke,Swansea,D,43,33
...,...,...,...,...,...,...
372,2016-05-15,Everton,Norwich,H,44,34
371,2016-05-15,Chelsea,Leicester,D,49,80
370,2016-05-15,Arsenal,Aston Villa,H,68,17
373,2016-05-15,Newcastle,Tottenham,H,34,70


In [11]:
def get_form(playing_stat, num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(playing_stat, num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h                 
    playing_stat['AM' + str(num)] = a

    
    return playing_stat


def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    print(playing_statistics.Date.iloc[-1].year, " OK!")
    return playing_statistics   


for i, dataset in enumerate(datasets):
    datasets[i] = add_form_df(dataset)

print("Finalizou.")

2001  OK!
2002  OK!
2003  OK!
2004  OK!
2005  OK!
2006  OK!
2007  OK!
2008  OK!
2009  OK!
2010  OK!
2011  OK!
2012  OK!
2013  OK!
2014  OK!
2015  OK!
2016  OK!
2017  OK!
2018  OK!
2019  OK!
2020  OK!
2021  OK!
Finalizou.


In [12]:
datasets[15].loc[:38, ["Date","HomeTeam", "AwayTeam","FTR",
                        "HM1", "HM2", "HM3", "HM4", "HM5",
                        "AM1", "AM2", "AM3", "AM4", "AM5",]].sort_values(by="Date")


,Date,HomeTeam,AwayTeam,FTR,HM1,HM2,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5
0,2015-08-08,Bournemouth,Aston Villa,A,M,M,M,M,M,M,M,M,M,M
1,2015-08-08,Chelsea,Swansea,D,M,M,M,M,M,M,M,M,M,M
2,2015-08-08,Everton,Watford,D,M,M,M,M,M,M,M,M,M,M
3,2015-08-08,Leicester,Sunderland,H,M,M,M,M,M,M,M,M,M,M
4,2015-08-08,Man United,Tottenham,H,M,M,M,M,M,M,M,M,M,M
5,2015-08-08,Norwich,Crystal Palace,A,M,M,M,M,M,M,M,M,M,M
6,2015-08-09,Arsenal,West Ham,A,M,M,M,M,M,M,M,M,M,M
7,2015-08-09,Newcastle,Southampton,D,M,M,M,M,M,M,M,M,M,M
8,2015-08-09,Stoke,Liverpool,A,M,M,M,M,M,M,M,M,M,M
9,2015-08-10,West Brom,Man City,A,M,M,M,M,M,M,M,M,M,M


In [13]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' , "HSA", "ASA", "HSTA", "ASTA"]

for i, dataset in enumerate(datasets):
    datasets[i] = dataset[cols]
    
print("Terminou.")

Terminou.


In [14]:
datasets[16].loc[:38, ["Date","HomeTeam", "AwayTeam","FTR",
                        "HM1", "HM2", "HM3", "HM4", "HM5",
                        "AM1", "AM2", "AM3", "AM4", "AM5",]].sort_values(by="Date")
datasets[20].columns.tolist()

['Date',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTGS',
 'ATGS',
 'HTGC',
 'ATGC',
 'HTP',
 'ATP',
 'HM1',
 'HM2',
 'HM3',
 'HM4',
 'HM5',
 'AM1',
 'AM2',
 'AM3',
 'AM4',
 'AM5',
 'HSA',
 'ASA',
 'HSTA',
 'ASTA']

In [15]:
def correct_name(playing_stat):
    playing_stat.HomeTeam = playing_stat.HomeTeam.replace({"Middlesboro": "Middlesbrough",
                                                           "Brighton": "Brighton & Hove Albion",
                                                           "Huddersfield": "Huddersfield Town"})
    playing_stat.AwayTeam = playing_stat.AwayTeam.replace({"Middlesboro": "Middlesbrough",
                                                           "Brighton": "Brighton & Hove Albion",
                                                           "Huddersfield": "Huddersfield Town"})

for dataset in datasets:
    dataset = correct_name(dataset)

In [16]:
standings = pd.read_csv(loc + "EPLStandindsFinal.csv")
standings.set_index(['Team'], inplace=True)
standings = standings.fillna(18) # Bota no 18 como o primeiro da lanterna, se o time subiu ele tá melhor q o 18, 
                                 # pelo menos, poderia botar 30 ou 21, mas talvez 18 se saia bem

def get_last(playing_stat, standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HomeTeamLP.append(standings.loc[ht][year])
        AwayTeamLP.append(standings.loc[at][year])
    playing_stat['HomeTeamLP'] = HomeTeamLP
    playing_stat['AwayTeamLP'] = AwayTeamLP
    print(playing_stat.Date.iloc[-1].year, " OK!")

    return playing_stat

for dataset in datasets:
    dataset = get_last(dataset, standings, dataset.Date.iloc[-1].year - 2001)

print("Finalizou.")

2001  OK!
2002  OK!
2003  OK!
2004  OK!
2005  OK!
2006  OK!
2007  OK!
2008  OK!
2009  OK!
2010  OK!
2011  OK!
2012  OK!
2013  OK!
2014  OK!
2015  OK!
2016  OK!
2017  OK!
2018  OK!
2019  OK!
2020  OK!
2021  OK!
Finalizou.


In [17]:
standings.loc[(standings.index == "Coventry")] # Vamos apenas até a liga de 2021, ou seja só usamos até 2020

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Team,,,,,,,,,,,,,,,,,,,,,
Coventry,14.0,19.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,...,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0


In [18]:
datasets[0].loc[:38, ["Date","HomeTeam", "AwayTeam","FTR",'HomeTeamLP','AwayTeamLP']].sort_values(by="Date")

,Date,HomeTeam,AwayTeam,FTR,HomeTeamLP,AwayTeamLP
0,2000-08-19,Charlton,Man City,H,18.0,18.0
1,2000-08-19,Chelsea,West Ham,H,5.0,9.0
2,2000-08-19,Coventry,Middlesbrough,A,14.0,12.0
3,2000-08-19,Derby,Southampton,D,16.0,15.0
4,2000-08-19,Leeds,Everton,H,3.0,13.0
5,2000-08-19,Leicester,Aston Villa,D,8.0,6.0
6,2000-08-19,Liverpool,Bradford,H,4.0,17.0
7,2000-08-19,Sunderland,Arsenal,H,7.0,2.0
8,2000-08-19,Tottenham,Ipswich,H,10.0,18.0
9,2000-08-20,Man United,Newcastle,H,1.0,11.0


In [19]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    print(playing_stat.Date.iloc[-1].year, " OK!")
    return playing_stat

for dataset in datasets:
    dataset = get_mw(dataset)

2001  OK!
2002  OK!
2003  OK!
2004  OK!
2005  OK!
2006  OK!
2007  OK!
2008  OK!
2009  OK!
2010  OK!
2011  OK!
2012  OK!
2013  OK!
2014  OK!
2015  OK!
2016  OK!
2017  OK!
2018  OK!
2019  OK!
2020  OK!
2021  OK!


In [20]:
datasets[1].loc[:38, ["Date","HomeTeam", "AwayTeam","FTR",'HomeTeamLP','AwayTeamLP', "MW"]].sort_values(by="Date")

,Date,HomeTeam,AwayTeam,FTR,HomeTeamLP,AwayTeamLP,MW
0,2001-08-18,Charlton,Everton,A,9.0,16.0,1
1,2001-08-18,Derby,Blackburn,H,17.0,18.0,1
2,2001-08-18,Leeds,Southampton,H,4.0,10.0,1
3,2001-08-18,Leicester,Bolton,A,13.0,18.0,1
4,2001-08-18,Liverpool,West Ham,H,3.0,15.0,1
5,2001-08-18,Middlesbrough,Arsenal,A,14.0,2.0,1
6,2001-08-18,Sunderland,Ipswich,H,7.0,5.0,1
7,2001-08-18,Tottenham,Aston Villa,D,12.0,8.0,1
9,2001-08-19,Man United,Fulham,H,1.0,18.0,1
8,2001-08-19,Chelsea,Newcastle,D,6.0,11.0,1


In [21]:
final_dataset = pd.concat(datasets)

final_dataset.reset_index(drop=True, inplace=True)

In [22]:
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

In [23]:
final_dataset['HTFormPtsStr'] = final_dataset['HM1'] + final_dataset['HM2'] + final_dataset['HM3'] + final_dataset['HM4'] + final_dataset['HM5']
final_dataset['ATFormPtsStr'] = final_dataset['AM1'] + final_dataset['AM2'] + final_dataset['AM3'] + final_dataset['AM4'] + final_dataset['AM5']

final_dataset['HTFormPtsStr3'] = final_dataset['HM1'] + final_dataset['HM2'] + final_dataset['HM3']
final_dataset['ATFormPtsStr3'] = final_dataset['AM1'] + final_dataset['AM2'] + final_dataset['AM3']

data = final_dataset

final_dataset['HTFormPts'] = final_dataset['HTFormPtsStr'].apply(get_form_points)
final_dataset['ATFormPts'] = final_dataset['ATFormPtsStr'].apply(get_form_points)

final_dataset['HTFormPts3'] = final_dataset['HTFormPtsStr3'].apply(get_form_points)
final_dataset['ATFormPts3'] = final_dataset['ATFormPtsStr3'].apply(get_form_points)

In [24]:
# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[:3] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[:3] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
final_dataset['HTWinStreak3'] = final_dataset['HTFormPtsStr'].apply(get_3game_ws)
final_dataset['HTWinStreak5'] = final_dataset['HTFormPtsStr'].apply(get_5game_ws)
final_dataset['HTLossStreak3'] = final_dataset['HTFormPtsStr'].apply(get_3game_ls)
final_dataset['HTLossStreak5'] = final_dataset['HTFormPtsStr'].apply(get_5game_ls)

final_dataset['ATWinStreak3'] = final_dataset['ATFormPtsStr'].apply(get_3game_ws)
final_dataset['ATWinStreak5'] = final_dataset['ATFormPtsStr'].apply(get_5game_ws)
final_dataset['ATLossStreak3'] = final_dataset['ATFormPtsStr'].apply(get_3game_ls)
final_dataset['ATLossStreak5'] = final_dataset['ATFormPtsStr'].apply(get_5game_ls)

In [25]:
# Get Goal Difference
final_dataset['HTGD'] = final_dataset['HTGS'] - final_dataset['HTGC']
final_dataset['ATGD'] = final_dataset['ATGS'] - final_dataset['ATGC']

# Diff in points
final_dataset['DiffPts'] = final_dataset['HTP'] - final_dataset['ATP']
final_dataset['DiffFormPts'] = final_dataset['HTFormPts'] - final_dataset['ATFormPts']
final_dataset['DiffFormPts3'] = final_dataset['HTFormPts3'] - final_dataset['ATFormPts3']

# Diff in last year positions
final_dataset['DiffLP'] = final_dataset['HomeTeamLP'] - final_dataset['AwayTeamLP']

In [26]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','HTP','ATP', "DiffPts"] 
# Tirei o 'DiffPts','DiffFormPts' da Lista por Sugestão do Gui
# FormPts são os das ultimas 5 partidas, DiffPts é o total
final_dataset.MW = final_dataset.MW.astype(float)

for col in cols:
    final_dataset[col] = final_dataset[col] / final_dataset.MW

In [27]:
# Não chequei as Differences, mas o resto sim, aparentemente tudo OK!
final_dataset.to_csv(loc + "premier_league2001-2021.csv", index = False)

final_dataset

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffFormPts3,DiffLP
0,2000-08-19,Charlton,Man City,4,0,H,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,0.0
1,2000-08-19,Chelsea,West Ham,4,2,H,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,-4.0
2,2000-08-19,Coventry,Middlesbrough,1,3,A,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,2.0
3,2000-08-19,Derby,Southampton,2,2,D,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,1.0
4,2000-08-19,Leeds,Everton,2,0,H,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0,0,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,2021-05-23,Liverpool,Crystal Palace,2,0,H,66,41,42,64,...,0,0,0,0,0.631579,-0.605263,0.578947,7,6,-13.0
7976,2021-05-23,Man City,Everton,5,0,H,78,47,32,43,...,0,0,0,0,1.210526,0.105263,0.631579,2,-1,-10.0
7977,2021-05-23,Sheffield United,Burnley,1,0,H,19,33,63,54,...,0,0,0,0,-1.157895,-0.552632,-0.500000,0,0,-1.0
7978,2021-05-23,West Ham,Southampton,3,0,H,59,47,47,65,...,0,0,0,0,0.315789,-0.473684,0.500000,0,-2,5.0
